In [1]:
from env import host, user, password
import acquire

In [2]:
df = acquire.get_zillow()

In [3]:
df.head()

,calculatedfinishedsquarefeet,bedroomcnt,bathroomcnt,taxvaluedollarcnt
0,3100.0,4.0,3.5,1023282.0
1,1465.0,2.0,1.0,464000.0
2,1243.0,3.0,2.0,564778.0
3,2376.0,4.0,3.0,145143.0
4,1312.0,3.0,3.0,119407.0
